# Forecasting Energy Demand using TimeGPT and N-HiTS

In [1]:
# Install the necessary libraries
!pip install -Uqq nixtla neuralforecast utilsforecast

# Import required libraries
import time
import requests
import pandas as pd
from nixtla import NixtlaClient
from utilsforecast.losses import mae, smape
from utilsforecast.evaluation import evaluate
from google.colab import userdata

# Set up API key for Nixtla
NIXTLA_API_KEY = userdata.get('NIXTLA_API_KEY')  # Replace with your actual API key if not in Colab
nixtla_client = NixtlaClient(api_key=NIXTLA_API_KEY)

# Define a function to load and process time-series data
def load_energy_data(url, start_date):
    """Load and preprocess energy demand data."""
    data = []
    start_date = pd.to_datetime(start_date)

    # Fetch data from URL
    response = requests.get(url)
    response.raise_for_status()

    # Process the time-series data
    for line in response.text.splitlines():
        if line.startswith('T'):
            parts = line.strip().split(':')
            unique_id = parts[0] + '-' + parts[1]
            values = list(map(float, parts[3].split(',')[:-1]))
            periods = len(values)
            timestamps = pd.date_range(start=start_date, periods=periods, freq='30min')

            for ts, val in zip(timestamps, values):
                data.append([unique_id, ts, val])

    return pd.DataFrame(data, columns=['unique_id', 'ds', 'y'])

# Load the energy demand dataset
url = 'https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/australian_electricity_demand_dataset.tsf'
start_date = '2002-01-01 00:00:00'
df = load_energy_data(url, start_date)

# Preview the data
print("Dataset Preview:")
print(df.groupby('unique_id').head(2))

# Visualize the dataset
print("\nPlotting the dataset...")
nixtla_client.plot(df, max_insample_length=365)

# Define train and test sets for forecasting
test_df = df.groupby('unique_id').tail(96)  # Last 2 days (96 half-hour intervals)
input_df = df.groupby('unique_id').apply(lambda group: group.iloc[-1104:-96]).reset_index(drop=True)  # Prior 21 days

# Perform forecasting with TimeGPT
print("\nForecasting with TimeGPT...")
start = time.time()
fcst_df = nixtla_client.forecast(
    df=input_df,
    h=96,                      # Forecast horizon of 96 intervals
    level=[90],                # 90% confidence interval
    finetune_steps=10,         # Fine-tuning steps for better accuracy
    finetune_loss='mae',       # Use Mean Absolute Error (MAE) as the loss function
    model='timegpt-1-long-horizon',
    time_col='ds',
    target_col='y',
    id_col='unique_id'
)
end = time.time()
timegpt_duration = end - start

print(f"Time taken by TimeGPT: {timegpt_duration:.2f} seconds")

# Plot TimeGPT forecast results
print("\nVisualizing TimeGPT forecast...")
nixtla_client.plot(test_df, fcst_df, models=['TimeGPT'], level=[90], time_col='ds', target_col='y')

# Evaluate TimeGPT forecast
print("\nEvaluating TimeGPT performance...")
fcst_df['ds'] = pd.to_datetime(fcst_df['ds'])
test_df = pd.merge(test_df, fcst_df, 'left', ['unique_id', 'ds'])

evaluation = evaluate(
    test_df,
    metrics=[mae, smape],
    models=["TimeGPT"],
    target_col="y",
    id_col='unique_id'
)

average_metrics = evaluation.groupby('metric')['TimeGPT'].mean()
print("TimeGPT Average Metrics:")
print(average_metrics)

# Perform forecasting with N-HiTS
print("\nForecasting with N-HiTS...")
from neuralforecast.core import NeuralForecast
from neuralforecast.models import NHITS

train_df = df.groupby('unique_id').apply(lambda group: group.iloc[-9696:-96]).reset_index(drop=True)  # Use last 202 days as training data

# Define N-HiTS model configuration
horizon = 96  # Forecast horizon
models = [NHITS(h=horizon, input_size=5*horizon, scaler_type='robust', batch_size=16, valid_batch_size=8)]
nf = NeuralForecast(models=models, freq='30min')

# Train and predict using N-HiTS
start = time.time()
nf.fit(df=train_df)
nhits_preds = nf.predict()
end = time.time()
nhits_duration = end - start

print(f"Time taken by N-HiTS: {nhits_duration:.2f} seconds")

# Evaluate N-HiTS forecast
print("\nEvaluating N-HiTS performance...")
preds_df = pd.merge(test_df, nhits_preds, 'left', ['unique_id', 'ds'])

evaluation = evaluate(
    preds_df,
    metrics=[mae, smape],
    models=["NHITS"],
    target_col="y",
    id_col='unique_id'
)

average_metrics_nhits = evaluation.groupby('metric')['NHITS'].mean()
print("N-HiTS Average Metrics:")
print(average_metrics_nhits)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.2/259.2 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 815.2/815.2 kB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 926.4/926.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 6.9 MB/s eta 0:00:00


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Dataset Preview:
       unique_id                  ds            y
0         T1-NSW 2002-01-01 00:00:00  5714.045004
1         T1-NSW 2002-01-01 00:30:00  5360.189078
230735    T2-VIC 2002-01-01 00:00:00  3535.867064
230736    T2-VIC 2002-01-01 00:30:00  3383.499028
461470    T3-QUN 2002-01-01 00:00:00  3382.041342
461471    T3-QUN 2002-01-01 00:30:00  3288.315794
693741     T4-SA 2002-01-01 00:00:00  1191.078014
693742     T4-SA 2002-01-01 00:30:00  1219.589472
924524    T5-TAS 2002-01-01 00:00:00   315.915504
924525    T5-TAS 2002-01-01 00:30:00   306.245864

Plotting the dataset...


<ipython-input-1-c00f6cbd7e30>:56: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  input_df = df.groupby('unique_id').apply(lambda group: group.iloc[-1104:-96]).reset_index(drop=True)  # Prior 21 days



Forecasting with TimeGPT...
Time taken by TimeGPT: 3.61 seconds

Visualizing TimeGPT forecast...

Evaluating TimeGPT performance...
TimeGPT Average Metrics:
metric
mae      436.787046
smape      0.075237
Name: TimeGPT, dtype: float64

Forecasting with N-HiTS...


...

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1000` reached.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

Time taken by N-HiTS: 936.50 seconds

Evaluating N-HiTS performance...
N-HiTS Average Metrics:
metric
mae      217.768283
smape      0.047968
Name: NHITS, dtype: float64


/usr/local/lib/python3.10/dist-packages/neuralforecast/core.py:214: FutureWarning: In a future version the predictions will have the id as a column. You can set the `NIXTLA_ID_AS_COL` environment variable to adopt the new behavior and to suppress this warning.
  warnings.warn(
